In [ ]:
%pip install -qU langchain openai langchain-elasticsearch tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.7/949.7 kB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 3.5 MB/s eta 0:00:00


In [11]:
# =====================================================
# 🚗 Electric Vehicle Type Predictor Chatbot (Gradio)
# =====================================================

!pip install -q pandas gradio

import pandas as pd
import re
import gradio as gr

# --- Load dataset ---
df = pd.read_csv("/content/Electric_Vehicle_Population_Data.csv")

# --- Preprocess ---
df['Make'] = df['Make'].astype(str).str.lower().str.strip()
df['Model'] = df['Model'].astype(str).str.lower().str.strip()
df['Electric Vehicle Type'] = df['Electric Vehicle Type'].astype(str).str.strip()
df['Electric Range'] = pd.to_numeric(df['Electric Range'], errors='coerce').fillna(0)

print(f"✅ Dataset loaded successfully! Shape: {df.shape}")

# --- Utility Functions ---

def normalize_text(text):
    """Normalize text for matching."""
    return re.sub(r'\s+', ' ', text.strip().lower())

def find_vehicle(make_model):
    """Find the closest matching vehicle in dataset."""
    key = normalize_text(make_model)
    match = df[(df['Make'] + ' ' + df['Model']).str.contains(key, case=False, regex=False)]
    return match if not match.empty else None

def predict_vehicle_type(make_model, user_range=None):
    """Predict BEV or PHEV using dataset info."""
    vehicle_data = find_vehicle(make_model)

    if vehicle_data is None:
        return f"Sorry, I couldn’t find '{make_model}' in the dataset. Please try another make and model."

    ev_type = vehicle_data['Electric Vehicle Type'].iloc[0]
    avg_range = vehicle_data['Electric Range'].mean()

    if user_range:
        if abs(user_range - avg_range) > 50:
            return (f"The range you mentioned ({user_range} miles) differs from the dataset's average ({avg_range:.0f} miles). "
                    f"However, according to records, the {make_model} is a **{ev_type}**.")
        else:
            description = "Battery Electric Vehicle (BEV), running solely on electricity." if "BEV" in ev_type else \
                          "Plug-in Hybrid Electric Vehicle (PHEV), combining electric and gasoline power."
            return (f"Based on your provided range ({user_range} miles) and the dataset, "
                    f"the {make_model} is a **{ev_type}** — a {description}")
    else:
        return f"According to the dataset, the {make_model} is classified as a **{ev_type}**."

# --- Chatbot Memory Variables ---
conversation_state = {"step": "greet", "make_model": None}

def chatbot(message, history):
    """Handle chat turns and maintain context."""
    global conversation_state
    message = message.strip()

    # Step 1: Greeting
    if conversation_state["step"] == "greet":
        conversation_state["step"] = "make_model"
        return "What’s the make and model of the electric vehicle you want me to look up?"

    # Step 2: Get make & model
    elif conversation_state["step"] == "make_model":
        conversation_state["make_model"] = message
        vehicle_data = find_vehicle(message)
        if vehicle_data is None:
            return "Sorry, I couldn’t find that make and model in the dataset. Please try another."
        conversation_state["step"] = "range"
        return f"Great! Can you tell me the approximate electric range (in miles) for the {message}?"

    # Step 3: Get range and predict
    elif conversation_state["step"] == "range":
        match = re.search(r'\d+', message)
        if not match:
            return "Please provide a valid numeric range, like '333 miles'."
        user_range = int(match.group())
        response = predict_vehicle_type(conversation_state["make_model"], user_range)
        conversation_state["step"] = "followup"
        return response + " Would you like to check another vehicle?"

    # Step 4: Follow-up
    elif conversation_state["step"] == "followup":
        if message.lower() in ["no", "n", "exit", "quit"]:
            conversation_state["step"] = "greet"
            return "Goodbye! 👋"
        else:
            conversation_state = {"step": "make_model", "make_model": None}
            return "Alright! What is the make and model of the next vehicle?"

# --- Gradio App with Initial Greeting ---
with gr.Blocks() as app:
    gr.Markdown("# ⚡ Electric Vehicle Type Predictor Chatbot")
    gr.Markdown("Chat with me to find out whether an electric vehicle is a **BEV** or **PHEV**, based on real dataset records.")

    chatbot_ui = gr.Chatbot(
        value=[[None, "Hi there! ⚡ I'm your EV Type Assistant. I can help you determine whether an electric vehicle is a BEV or PHEV"]],
        height=400
    )
    user_input = gr.Textbox(label="Your message", placeholder="Type here...")
    clear = gr.Button("Clear Chat")

    def user_interact(message, history):
        response = chatbot(message, history)
        history.append((message, response))
        return history, ""

    user_input.submit(user_interact, [user_input, chatbot_ui], [chatbot_ui, user_input])
    clear.click(lambda: [[None, "Hi there! ⚡ I'm your EV Type Assistant. I can help you determine whether an electric vehicle is a BEV or PHEV"]], None, chatbot_ui)

app.launch(share=True)


✅ Dataset loaded successfully! Shape: (264628, 17)


/tmp/ipython-input-2263885666.py:102: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot_ui = gr.Chatbot(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b25c3ded2ba6024ddb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
